## Universidad Nacional de Colombia

## Diplomado Ciencia de Datos 


## Validación K-means

En este caso práctico continuaremos con el caso de clustering que hicimos la clase anterior. Haremos la validación de nuestra aplicación de k-means.

El caso estará estructurado así
1. Resumir el caso anterior
2. Ajustar y comparar distintos k-means
3. Revisar criterios de validación
4. Hacer conclusiones a partir del análisis

**Contexto:** Las competencias deportivas cada día recogen una gran cantidad de datos relacionados con el desempeño de sus equipos y jugadores para encontrar patrones en estos datos y tomar decisiones informadas basadas en ellos. De esta manera la competencia aumenta tanto dentro como fuera de la cancha

**Problema de negocio:** Se tienen los datos de desempeño de los equipos de baloncesto del torneo NCAA March Madness que contiene las estadísticas de juego de 353 equipos de la liga. El objetivo es inspeccionar esta data utilizando técnicas de visualización y agrupación para encontrar patrones en el desempeño de los equipos y generar recomendaciones de umbrales en las estadísticas para que un equipo esté en el grupo de desempeño superior.

In [ ]:
# librerías
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn
from scipy.stats import norm
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, StandardScaler
from sklearn.metrics import silhouette_samples,silhouette_score
from scipy.spatial.distance import cdist
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# display setting Para visualizar el máximo de columnas
pd.set_option('display.max_columns', None)

In [ ]:
datos = pd.read_csv('basketball_19.csv')

Estas son las variables que contiene el conjunto de datos 

- TEAM: Equipo
- CONF: La conferencia en la que el equipo participa(A10 = Atlantic 10, ACC = Atlantic Coast Conference, AE = America East, Amer = American, ASun = ASUN, B10 = Big Ten, B12 = Big 12, BE = Big East, BSky = Big Sky, BSth = Big South, BW = Big West, CAA = Colonial Athletic Association, CUSA = Conference USA, Horz = Horizon League, Ivy = Ivy League, MAAC = Metro Atlantic Athletic Conference, MAC = Mid-American Conference, MEAC = Mid-Eastern Athletic Conference, MVC = Missouri Valley Conference, MWC = Mountain West, NEC = Northeast Conference, OVC = Ohio Valley Conference, P12 = Pac-12, Pat = Patriot League, SB = Sun Belt, SC = Southern Conference, SEC = South Eastern Conference, Slnd = Southland Conference, Sum = Summit League, SWAC = Southwestern Athletic Conference, WAC = Western Athletic Conference, WCC = West Coast Conference)
- G: Número de partidos jugados
- W: Número de partidos ganados
- ADJOE: Estimación de eficiencia ofensiva, puntos anotados por cada 100 posesiones
- ADJDE: Estimación de eficiencia defensiva, puntos permitidos por cada 100 posesiones del equipo contrario
- BARTHAG: Probabilidad de vencer a un equipo
- EFG_O: Effective Field Goal Percentage Shot
- EFG_D: Effective Field Goal Percentage Allowed
- TOR: Porcentaje de rotación permitida (equipo pierde la posesión del balón contra el equipo contrario antes de que un jugador dispare a la canasta de su equipo)
- TORD: Porcentaje de rotación hecha al equipo contrario (se roba la pelota al contrincante)
- ORB: Porcentaje de rebote ofensivo
- B: Porcentaje de rebote defensivo
- FTR : Tasa de tiros libres hechos(que hace el equipo)
- FTRD: Tasa de tiros libres permitidos (que hace el contrincante)
- 2P_O: Porcentaje de tiros de 2 puntos hechos
- 2P_D: Porcentaje de tiros de 2 puntos permitidos
- 3P_O: Porcentaje de tiros de 3 puntos hechos
- 3P_D: Porcentaje de tiros de 3 puntos permitidos
- ADJ_T: Posesión del balón por 40 min
- WAB: Triunfos por encima de la 'burbuja' (la burbuja es el límite definido para pasar al campeonato NCAA March Madness Tournament
- POSTSEASON: Ronda en la que el equipo de fue eliminado (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)
- SEED: Semilla definida por el torneo


In [ ]:
datos.head()

## Exploración de los datos 

En el caso pasado hicimos una exploración general de las variables 

In [ ]:
# Partidos jugados
plt.figure(figsize=(15,15))

plt.subplot(321)
plt.scatter(y=datos['BARTHAG'], x=datos['ADJOE'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Eficiencia Ofensiva', fontsize=16)

plt.subplot(322)
plt.scatter(y=datos['BARTHAG'], x=datos['EFG_O'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('% Tiros efectivos', fontsize=16)


plt.subplot(323)
plt.scatter(y=datos['BARTHAG'], x=datos['ORB'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Porcentaje de rebote ofensivo', fontsize=16)


# Partidos ganados
plt.subplot(324)
plt.scatter(y=datos['BARTHAG'], x=datos['TOR'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob Ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Porcentaje rotación', fontsize=16)


plt.subplot(325)
plt.scatter(y=datos['BARTHAG'], x=datos['2P_O'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Porcentaje de tiros de 2 puntos hechos', fontsize=16)


plt.subplot(326)
plt.scatter(y=datos['BARTHAG'], x=datos['ADJ_T'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Posesión del balón', fontsize=16)


plt.show()

Con ésta exploración inicial seleccionamos algunas variables para hacer el análisis de clusters

In [ ]:
# sacamos del análisis variables categóricas
km_data = datos.drop(['TEAM','CONF','POSTSEASON','SEED'],axis=1)

In [ ]:
# seleccionamos las variables para el análisis
km = km_data[['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB']]
km.head()

## Agrupamiento 

Al igual que en ACP es importante estandarizar las variables que vamos a utilizar. La función **StandardScaler** nos permite hacerlo en una sola linea

In [ ]:
scaler = StandardScaler()
km_scale = scaler.fit_transform(km)

En el caso anterior definimos el número de clusters como $k=3$ para el algortimo de kmeans. Al ajustar el agrupamiento jerárquico observabamos que otros posibles números de cluster podríamos utilizar 

In [ ]:
plt.figure(figsize=(10,8))
dendrogram = sch.dendrogram(sch.linkage(km_scale, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Equipos')
plt.ylabel('Distancias euclideanas')
plt.show()

Podemos entonces ajustar el kmeans con $k=3$. Adicionalmente, tiene mucha lógica ajustarlo con $k=4$ o $k=5$

In [ ]:
kmeans_3k = KMeans(init="random",n_clusters=3,n_init=10,max_iter=300,random_state=42)
kmeans_3k.fit(km_scale)

In [ ]:
kmeans_4k = KMeans(init="random",n_clusters=4,n_init=10,max_iter=300,random_state=42)
kmeans_4k.fit(km_scale)

## Validación

Emplearemos validación interna para comparar las soluciones anteriores. Para esto podemos evaluar la **cohesión** revisando el argumento **inertia_** que es la suma de distancias cuadradas de cada punto a su centroide

In [ ]:
print(kmeans_3k.inertia_)

In [ ]:
print(kmeans_4k.inertia_)

Recordemos que entre menor sea esta suma de cuadrados será mejor la solución. Sin embargo siempre que se generen más clusters este número tiende a disminuir. Por esta razón debemos enocontrar un punto de balance entre tener un número bajo de inertia_ y un número adecuado de clusters

Podemos también revisar los coeficientes de silhouette para ver qué tan bien asignado está cada punto a su cluster

In [ ]:
score = silhouette_score(km_scale, kmeans_3k.labels_)
print('Silhouetter Score 3 clusters: %.3f' % score)

In [ ]:
import matplotlib.cm as cm
#modified code from http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

def silplot(X, clusterer, pointlabels=None):
    cluster_labels = clusterer.labels_
    n_clusters = clusterer.n_clusters
    
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(11,8.5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters = ", n_clusters,
          ", the average silhouette_score is ", silhouette_avg,".",sep="")

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(0,n_clusters+1):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=200, lw=0, alpha=0.7,
                c=colors, edgecolor='k')
    xs = X[:, 0]
    ys = X[:, 1]
    
    if pointlabels is not None:
        for i in range(len(xs)):
            plt.text(xs[i],ys[i],pointlabels[i])

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % int(i), alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

In [ ]:
silplot(km_scale, kmeans_3k)

Con 4 clusters

In [ ]:
score = silhouette_score(km_scale, kmeans_4k.labels_)
print('Silhouetter Score 4 clusters: %.3f' % score)

In [ ]:
silplot(km_scale, kmeans_4k)

Hay menor número de puntos "mal representados" en la solución de 3 clusters

## Elección de k

Utilizaremos la comparación de valores de suma de cuadrados dentro total con respecto a diferentes valores de $k$. Este método es conocido como el **método del codo**

In [ ]:
ssw = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k,init="random",n_init=10,max_iter=300,random_state=42)
    kmeanModel.fit(km_scale)
    ssw.append(kmeanModel.inertia_)
    
plt.figure(figsize=(12,8))
plt.plot(K, ssw, 'bx-')
plt.xlabel('Número de clusters $k$')
plt.ylabel('Suma de cuadrados dentro')
plt.title('El método del codo mostrando el k óptimo')
plt.show()

## ¿ Cuál consideran sería el valor óptimo para k según el método del codo?

Adicionalmente, podemos revisar el coeficiente de Silhouette

In [ ]:
scores = [0]
for k in range(2,11):
    fitx = KMeans(n_clusters=k,init="random",n_init=10,max_iter=300,random_state=42).fit(km_scale)
    score = silhouette_score(km_scale, fitx.labels_)
    scores.append(score)
    
plt.figure(figsize=(11,8.5))
plt.plot(range(1,11), np.array(scores), 'bx-')
plt.xlabel('Número de clusters $k$')
plt.ylabel('Average Silhouette')
plt.title('Método Silhouette mostrando el $k$ óptimo')
plt.show()

## ¿Cuál consideran sería el valor óptimo para k según el método de silhouette?

No necesariamente los métodos deben coincidir ni dan una única respuesta. Depende del investigador y el contexto analizar qué solución podría ser mejor

## Validación estructura de clusters

Podemos guardar los clusters asignados por el algoritmo cuando se usaron $k=3$ y $k=4$ y evaluar las diferencias entre ellos en cómo están definidos en la data

In [ ]:
datos['cluster_3k'] = kmeans_3k.labels_
datos['cluster_4k'] = kmeans_4k.labels_

Revisemos la distribución de estos clusters

In [ ]:
print(datos.cluster_3k.value_counts())
datos.cluster_3k.value_counts().plot(kind='bar')

In [ ]:
print(datos.cluster_4k.value_counts())
datos.cluster_4k.value_counts().plot(kind='bar')

Para tener mayor información podríamos visualizar las variables originales de acuerdo a los clusters para entender mejor la estructura

**Solución 3 clusters**

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
colors ={0: 'blue', 1: 'pink', 2: 'orange' , 3:'green', 4:'red'}
plt.scatter(y=datos['BARTHAG'], x=datos['ADJOE'],c=datos['cluster_3k'].apply(lambda x: colors[x]),alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Efic. Ofensiva', fontsize=16)

plt.subplot(122)
plt.scatter(y=datos['BARTHAG'], x=datos['EFG_O'],c=datos['cluster_3k'].apply(lambda x: colors[x]),alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('% Tiros efectivos', fontsize=16)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p3d = ax.scatter(xs=datos['BARTHAG'], ys=datos['ADJOE'], zs=datos['EFG_O'], s=30, c=datos['cluster_3k'], cmap = cm.coolwarm)
plt.show()

**Solución 4 clusters**

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
colors ={0: 'blue', 1: 'pink', 2: 'orange' , 3:'green', 4:'red'}
plt.scatter(y=datos['BARTHAG'], x=datos['ADJOE'],c=datos['cluster_4k'].apply(lambda x: colors[x]),alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Efic. Ofensiva', fontsize=16)

plt.subplot(122)
plt.scatter(y=datos['BARTHAG'], x=datos['EFG_O'],c=datos['cluster_4k'].apply(lambda x: colors[x]),alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('% Tiros efectivos', fontsize=16)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p3d = ax.scatter(xs=datos['BARTHAG'], ys=datos['ADJOE'], zs=datos['EFG_O'], s=30, c=datos['cluster_4k'], cmap = cm.coolwarm)
plt.show()

In [ ]:
pd.crosstab(datos['cluster_3k'],datos['cluster_4k'])

En los anteriores gráficos se observa cómo claramente al aumentar el número de clusters, el nuevo grupo se traslapa con los anteriores. Esta situación podría llevar a una solución redundante donde dos clusters van a tener características muy similares o cercanas. Revisemos la caracterización de los clusters con las variables numéricas

In [ ]:
datos.groupby('cluster_3k')['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB'].mean()

In [ ]:
datos[['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB']].mean()

In [ ]:
datos.groupby('cluster_4k')['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB'].mean()

En la solución de 4 clusters se puede ver como el cluster de nivel medio en la solución original es ahora reemplazado por dos clusters que podríamos considerar de desempeño medio-bajo (3) y medio-alto (1)

## Comparación ACP y Kmeans

Ajustamos un ACP con las variables de interés

In [ ]:
pca = PCA(n_components=5)
pca_bs = pca.fit(km_scale)
pca_bs = pca.transform(km_scale)

Revisemos cuántos componentes deberíamos utilizar

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(np.array([1,2,3,4,5]),pca.explained_variance_ratio_)
ax.set(xlabel = "Dimension",
       ylabel = "Porcentaje de varianza explicada")
plt.show()

In [ ]:
principalDf = pd.DataFrame(data = pca_bs, columns = ['PC1', 'PC2','PC3', 'PC4','PC5'])
principalDf.head()

In [ ]:
datos['CP1'] = principalDf['PC1']
datos['CP2'] = principalDf['PC2']
datos['CP3'] = principalDf['PC3']

Evaluemos cuántos clusters deberíamos tener con estas nuevas variables 

In [ ]:
ssw = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k,init="random",n_init=10,max_iter=300,random_state=42)
    kmeanModel.fit(datos[['CP1','CP2','CP3']])
    ssw.append(kmeanModel.inertia_)
    
plt.figure(figsize=(12,8))
plt.plot(K, ssw, 'bx-')
plt.xlabel('Número de clusters $k$')
plt.ylabel('Suma de cuadrados dentro')
plt.title('El método del codo mostrando el k óptimo')
plt.show()

Nuevamente parece que k=4 podría ser un valor apropiado para el número de clusters

In [ ]:
kmeans_acp = KMeans(init="random",n_clusters=4,n_init=10,max_iter=300,random_state=42)
kmeans_acp.fit(datos[['CP1','CP2','CP3']])

**Comparación de resultados**

Cohesión 

In [ ]:
print(kmeans_4k.inertia_)
print(kmeans_acp.inertia_)

Silhouette

In [ ]:
score = silhouette_score(datos[['CP1','CP2','CP3']], kmeans_acp.labels_)
print('Silhouetter Score clusters ACP: %.3f' % score)

In [ ]:
score = silhouette_score(km_scale, kmeans_4k.labels_)
print('Silhouetter Score 4 clusters: %.3f' % score)

In [ ]:
silplot(datos[['CP1','CP2','CP3']].values,kmeans_acp)

In [ ]:
datos['cluster_acp'] = kmeans_acp.labels_

In [ ]:
print(datos.cluster_acp.value_counts())
datos.cluster_acp.value_counts().plot(kind='bar')

In [ ]:
pd.crosstab(datos['cluster_4k'],datos['cluster_acp'])

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
colors ={0: 'blue', 1: 'pink', 2: 'orange' , 3:'green', 4:'red'}
plt.scatter(y=datos['BARTHAG'], x=datos['ADJOE'],c=datos['cluster_acp'].apply(lambda x: colors[x]),alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Efic. Ofensiva', fontsize=16)

plt.subplot(122)
plt.scatter(y=datos['BARTHAG'], x=datos['EFG_O'],c=datos['cluster_acp'].apply(lambda x: colors[x]),alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('% Tiros efectivos', fontsize=16)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p3d = ax.scatter(xs=datos['BARTHAG'], ys=datos['ADJOE'], zs=datos['EFG_O'], s=30, c=datos['cluster_acp'], cmap = cm.coolwarm)
plt.show()

## Ejercicios

1. Ajustemos un nuevo kmeans considerando ahora 5 clusters
2. ¿Qué diferencias se evidencian en comparación con los dos anteriores?
3. Analicemos la caracterización de los 5 clusters con las variables originales, ¿qué nombre le podríamos dar a cada uno de los clusters?
4. Después de analizar estas tres opciones, ¿cuál sería la mejor solución en este caso?

## Conclusiones

- Efectivamente se logran diferenciar tres grandes segmentos en los equipos: desempeño bajo, medio y alto
- Las métricas de validación son una gran herramienta para tomar la decisión sobre el número de clusters a utilizar
- También es necesario tener en cuenta el contexto de los datos para definir si la solución tiene sentido
